# Load previously exported files

The file `extra_features.txt` contains important property information like number and quality of pools, detached garages, outbuildings, canopies, and more. Let's load this file and grab a subset with the important columns to continue our study.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import pickle

import pandas as pd

from src.definitions import ROOT_DIR
from src.data.utils import Table, save_pickle

In [3]:
building_res_comps_fn = ROOT_DIR / 'data/raw/2016/building_res_comps.pickle'
assert building_res_comps_fn.exists()

with open(building_res_comps_fn, 'rb') as f:
    building_res_comps = pickle.load(f)

In [12]:
fixtures_comps_fn = ROOT_DIR / 'data/raw/2016/fixtures_comps.pickle'
assert fixtures_comps_fn.exists()

with open(fixtures_comps_fn, 'rb') as f:
    fixtures_comps = pickle.load(f)

In [5]:
real_acct_comps_fn = ROOT_DIR / 'data/raw/2016/real_acct_comps.pickle'
assert real_acct_comps_fn.exists()

with open(real_acct_comps_fn, 'rb') as f:
    real_acct_comps = pickle.load(f)

In [13]:
extra_features_count_grade_fn = ROOT_DIR / 'data/raw/2016/extra_features_count_grade.pickle'
assert extra_features_count_grade_fn.exists()

with open(extra_features_count_grade_fn, 'rb') as f:
    extra_features_count_grade = pickle.load(f)

# Merge tables

In [14]:
extra_features_count_grade.head()

dscr,acct,Canopy_count,Carport_count,Foundation Defect_count,Garage_count,Outbuildings_count,Pools_count,Canopy_grade,Carport_grade,Foundation Defect_grade,Garage_grade,Outbuildings_grade,Pools_grade
0,21440000001,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN
1,21650000007,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,4.0
2,21700000013,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN
3,21750000013,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN
4,22070000007,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN


In [15]:
comps = building_res_comps.merge(fixtures_comps,
                                 how='inner',
                                 on='acct',
                                 validate='one_to_one')

In [17]:
comps = comps.merge(real_acct_comps,
                    how='inner',
                    on='acct',
                    validate='one_to_one')

In [19]:
comps = comps.merge(extra_features_count_grade,
                    how='inner',
                    on='acct',
                    validate='one_to_one')

In [20]:
comps.head()

,acct,dscr,date_erected,yr_remodel,im_sq_ft,act_ar,heat_ar,gross_ar,eff_ar,base_ar,...,Foundation Defect_count,Garage_count,Outbuildings_count,Pools_count,Canopy_grade,Carport_grade,Foundation Defect_grade,Garage_grade,Outbuildings_grade,Pools_grade
0,21700000013,Low,1920,2004,1434,1622,1434,1622,1453,1434,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN
1,22080000008,Low,1940,0,2240,2732,2240,2732,2270,2240,...,NaN,2.0,2.0,NaN,NaN,NaN,NaN,5.0,4.0,NaN
2,22650000005,Average,1936,0,2232,2638,2232,2638,2231,2232,...,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN
3,22660000014,Average,1895,0,1277,1277,1277,1277,1277,1277,...,NaN,NaN,NaN,NaN,NaN,4.5,NaN,NaN,NaN,NaN
4,30390000013,Low,1939,0,1348,1612,1348,1612,1419,1348,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN


# Export comps

In [21]:
save_fn = ROOT_DIR / 'data/interim/2016/comps.pickle'
save_pickle(comps, save_fn)